# Convert individual z-layers and channels to stacks

Individual images for z-layers and channels are combined to one stack per sample

## Requirements
- A folder with images that should be converted

## Config

### The following code imports and declares functions used for the processing:

In [ ]:
#################################
#  Don't modify the code below  #
#################################

import os
import re
import time
import numpy as np
import intake_io
from tqdm import tqdm
from am_utils.utils import walk_dir

## Specify data paths and analysis parameters

### Please provide data paths:

`input_dir`: folder with images of cells to be segmented

`output_dir`: folder to save results

In [ ]:
input_dir = "/research/sharedresources/cbi/data_exchange/kriwagrp/05282021_CoLocalization"
output_dir = "/research/sharedresources/cbi/common/Anna/test/input"

### The following code lists all image files in the input directory:

In [ ]:
#################################
#  Don't modify the code below  #
#################################

samples = walk_dir(input_dir)

print(f'{len(samples)} images were found:')
print(np.array(samples))

### Please specify codes for channel and z-position:

Specify the sequence of characters that precedes the channel and z-position numbering, including the separator (e.g. "_") that precedes the channel code.

For example, if the image name is "my_experiment_position1_C0_z001.tif", the codes should be as follows:

`channel_code` = "_C"

`z_position_code` = "_z"


In [ ]:
channel_code = "_C"
z_position_code = "_Z"

### The following code sorts channels and z-positions into 3D stacks and extracts voxel size

In [ ]:
#################################
#  Don't modify the code below  #
#################################

sources = []
tags = []
# try different order of channel and z tags to extract the shorted i tag
for code in [channel_code, z_position_code]:
    tag = {}
    if channel_code is not None:
        tag['c'] = channel_code
    if z_position_code is not None:
        tag['z'] = z_position_code
    if channel_code is not None and z_position_code is not None:
        tag['i'] = re.compile(rf"{input_dir}(.*){code}\d+")
    tags.append(tag)
    sources.append(intake_io.source.FilePatternSource(input_dir, 
                                                      axis_tags=tag,
                                                      extensions=['.' + samples[0].split('.')[-1]], 
                                                      include_filters=[], 
                                                      exclude_filters=[],
                                                      ))
    
# select the source corresponding to the shortest base string
if 'i' in sources[0]._files.files.columns:
    fns = np.array([src._files.files.iloc[0]["i"] for src in sources])

    minind = np.where(fns == min(fns, key=len))[0][0]
else:
    minind = 0
    
src = sources[minind]
tag = tags[minind]
npartitions = intake_io.imload(src, metadata_only=True)['npartitions']

print(fr'{npartitions} stacks found')

# extract voxel size
print('Extracting metadata...')
coords = dict({'x': None, 'y': None, 'z': None})
img = intake_io.imload(src, partition=0)
for key in coords.keys():
    if key in img.coords:
        coords[key] = img.coords[key].data[1]
            
print('The following voxel sizes were detected:')
for key in coords.keys():
    print(rf'{key}: {coords[key]}')
        
print('\n The following channels have been detected:')
print(img['c'].data)


### Please specify correct voxel size 

Keep `None`, if the value loaded from the dataset is correct

In [ ]:
x = None
y = None
z = 0.2

### Please specify channels names

If you'd like to relabel channels, specify channel names as an array (e.g. ['channel_1', 'channel_2']). Specify `None` to keep the default channel labels.

In [ ]:
channel_names = ['DNA', 'GFP', 'mCherry']
# channel_names = ['GFP']

### The following code relabels channels and reassigns voxel size

In [ ]:
#################################
#  Don't modify the code below  #
#################################

vs = [x, y, z]
for i, c in enumerate(['x', 'y', 'z']):
    if vs[i] is not None:
        coords[c] = vs[i]

src = intake_io.source.FilePatternSource(input_dir,
                                        axis_tags=tag,
                                        extensions=['.' + samples[0].split('.')[-1]],
                                        metadata={"spacing": coords,
                                                  "coords": {'c': channel_names}})

### The following code load a random image/stack

In [ ]:
#################################
#  Don't modify the code below  #
#################################

dataset = intake_io.imload(src, partition=np.random.randint(npartitions))
print(dataset)

### The following code saves the combined stacks into the output folder

In [ ]:
#################################
#  Don't modify the code below  #
#################################

fns = src._files.files['i'].unique()
for i in tqdm(range(npartitions)):
    dataset = intake_io.imload(src, coords=coords, partition=i)
    fn = output_dir + fns[i].replace(' ', '_') + '.tif'
    os.makedirs(os.path.dirname(fn), exist_ok=True)
    intake_io.imsave(dataset, fn)
